# Phys 581 Winter 2019
# Final Project: Computational methods in quantum many-body physics
## Alexander Hickey, 10169582

### Introduction

Over the past two-decades, the increasing computational power available to researchers has lead to significant advances in the area of quantum many-body physics.